# Datapipeline introduksjon

# Workshop: Introduksjon til dataflyt og transformasjon

**Du vil lære:**
- Helt overordnet hva dataflyt og transformasjon er, hva det innebærer og hvordan det utføres i praksis
- Litt om Google Cloud Storage og Google BigQuery
- Å laste inn, hente ut og jobbe med data fra Google BigQuery
- Gjøre enkle transformasjoner ved hjelp av datamanipulerings verktøy
- Lage et nytt og rikere datasett med data fra flere kilder 

**Du vil _ikke_ lære:**
- Hvordan man setter opp en faktisk dataflyt eller lignende i Google Cloud Platform
- Spesifikke detaljer om verktøy som Google Cloud Platform, Pandas, Matplotlib, Keras, Tensorflow og liknende


## Noen utvalgte Google skykomponenter
#### Storage Bucket (Google Cloud Storage)

En storage bucket er fin å ha når man ønsker å lagre store mengder med rådata, eller ustrukturerte data. Hvis man har f.eks CSV uttrekk, kan disse lastes opp i en cloud storage.

Cloud Storage er også fin å bruke som et landingsområde for dataene dine. F.eks hvis du henter data via et API, kø/streaming eller batch, men ikke vet hvordan datastrukturene dine ser ut, kan du lagre de på Cloud Storage. På denne måten trenger du ikke å være avhengig av kildesystemet hvor dataene stammaer fra for å jobbe videre med disse.

En av fordelene med Google Cloud Storage er at det er billig å lagre store mengder data.

Det er derimot ikke så lett å lese innholdet i filene direkte fra en storage bucket. Vi ønsker derfor å flytte disse et sted hvor det er lett å analysere. Et slikt verktøy kan være Google BigQuery, som er en database tilpasset analyse. Dette kan videre kobles opp mot visualiseringsverktøy som Google Data Studio eller Google Colab notebooks (ML).

#### Google BigQuery

BigQuery er en SQL basert database som er optimalisert for analyse. I motsetning til tradisjonelle SQL servere, er den kolonnebasert (ikke radbasert). Dette medfører at den er veldig rask på å regne ut aggregerte tall, som er supert for analyseformål. Den takler også brede tabeller veldig godt, slik at man kan ha mange flagg/attributter per rad.



#### Oppgave 1 - Google Cloud Storage
I Google Cloud Console (GUI) for prosjektet (data-intro), finn Google Cloud Storage. Her finner du en bucket med to ulike datasett. Hva finner du ut om disse datasettene (metadata)?
- Hvilken filtype er de?
- Hvor store er filene?
- Annen informasjon?







#### Oppgave 2 - BigQuery
I Google Cloud Storage kan vi se "data om data(settet)" vårt, såkalt metadata. Men det er vanskelig å se hva som faktisk ligger inne i datasettet. Dette er ikke lett å se fra en Storage Bucket, så vi ønsker å flytte dataen til et annet verktøy.


Vi ønsker å flytte datasettene til BigQuery for å kunne se innholdet.

1. Finn BigQuery i menyen
2. Velg prosjektet "Data intro" og deretter marker datasettet bysykkel_main
3. I menylinjen oppe til høyre, velg "Create table". Kall den nye tabellen din bysykkel_(gruppenavn)
4. Her kan du velge datakilden din. Vi ønsker å velge oslobysykkel datasettet fra Storage Bucket. Filformatet fant du i oppgave 1. Fyll inn informasjon om "destination". La BigQuery definere skjema for deg, og behold ellers standard innstillingene.

Datasettet er nå lastet inn i BigQuery. BigQuery har en preview funksjon, i tillegg til muligheten til å spørre om data ved SQL-spørringer. Hva finner du ut om skjema og innholdet?




#### Oppgave 3 - BigQuery
Gjenta det samme for det andre datasettet, kall tabellen værdata_(gruppenavn).



#### Oppgave 4 - Utvidet datasett
Vi ønsker å slå sammen de to datasettet slik at vi kan gjøre analyse på tvers disse senere.

Når vi slår sammen data gjør vi en *transformasjon* på datasettet, det vil si vi gjør noen endringer på datasettet for å øke kvaliteten for analyse.

Det finnes flere verktøy man kan benytte for dette, i denne workshopen bruker vil notebook-verktøyet Google Colab.

💡 **NB! Det fungerer ikke så bra å redigere og/eller kjøre kode i Colab samtidig med andre brukere. Det er derfor viktig at du jobber i din egen notebook, og ikke i denne hvor oppgavene står.**

1. Opprett en Colab notebook. Du kan kalle denne hva du vil. Eventuelt kan du lage en kopi av denne notebooken.
2. Koble deg til BigQuery:
(Kopier kodesnuttene under inn i din notebook).
  - Først må du autentisere deg (med din Bekk Google bruker som har tilgang til BigQuery)


In [ ]:
# Authenticate your Google Account
# Doing so means you have access to various
# resources connected to your account, such
# as BigQuery tables, Storage buckets etc.
from google.colab import auth
auth.authenticate_user()


3.  Når du er autentisert, laster du inn bysykkel datasettet og værdatasettet til gruppen din inn i notebooken deres. Datasettene blir lastet inn som dataframes ( les mer om Dataframes her https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). Du kan kalle dataframesene dine henholdsvis `df_bysykkel` og `df_weather`.

<details><summary>🚨 Løsningsforslag</summary>

Husk å kjøre kodesnutten med importene og hjelpefunksjonen før du kjører løsningsforslaget under:

```# Løsning 
df = load_bigquery_data("data-intro", "bysykkel_main", "bysykkel_[DITT_GRUPPENAVN]")
df.head()
```
```
df_weather = load_bigquery_data("data-intro", "bysykkel_main", "værdata_joakim")
df_weather.head()
```

</details>


In [ ]:
# HUSK Å KOPIER DENNE KODEBLOKKEN TIL NOTEBOOKEN DIN, OG KJØR DEN FOR Å FÅ TILGANG TIL IMPORTENE OG HJELPEFUNKSJONEN
# Importer eksterne avhengigheter
from google.cloud import bigquery_storage, bigquery
from google.cloud.bigquery_storage import types
from google.cloud.bigquery_storage_v1 import enums
import pandas

# Input:
#   project_id: string
#   dataset_id: string
#     table_id: string
# 
# Output:
#   dataframe: pandas dataframe
#

# Hjelpemetode for å laste inn datasettet ditt. Du kan bruke denne for å laste inn datasettene dine
def load_bigquery_data(project_id, dataset_id, table_id):
    bqstorageclient = bigquery_storage.BigQueryReadClient()
    table = f"projects/{project_id}/datasets/{dataset_id}/tables/{table_id}"
    parent = "projects/{}".format(project_id)

    # Opprett en read-session mot en tabell i BigQuery
    requested_session = types.ReadSession(
        table=table,
        data_format = enums.DataFormat.ARROW
    )
    read_session = bqstorageclient.create_read_session(
        parent=parent,
        read_session=requested_session,
        max_stream_count=1,
    )

    # Les data fra BigQuery, putt i en liste med "frames"
    # og kombinér til én enkelt Pandas DataFrame
    stream = read_session.streams[0]
    reader = bqstorageclient.read_rows(stream.name)
    frames = []
    for message in reader.rows(read_session).pages:
        frames.append(message.to_dataframe())
    dataframe = pandas.concat(frames)

    return dataframe

3. Vi må finne en kolonne med fellesdata for å kunne slå de sammen (joine) tabellene. Så du noen fellesnevnere da du undersøkte innholdet i tabellene?

4. Et alternativ er å slå sammen tabellene basert på dato. Værdata har en entry per rad basert på dato. På bysykkel datasettet har vi flere kolonner som inneholder dato, så her må vi velge en. Valget avhenger av hva vi ønsker å analysere. Dette datasettet er ikke så omfattende, så i vårt eksempel velger vi å joine på turens starttidspunkt (`started_at`).

- Slå sammen tabellene ved å bruke `started_at` i bysykkeldatasettet og `date` i værdatasettet.

  - 💡  **Tips:** [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) funksjonen i pandas kan hjelpe deg med å slå sammen datasett.


<details><summary>🚨 Løsningsforslag</summary>
Vi kan slå sammen datasett ved å bruke `merge`, og fortelle funksjonen hvilke kolonner som skal slås sammen, samt hvilken type join vi ønsker (f.eks left, right, outer etc.)

```
  df_2 = pandas.merge(df, df_weather, on=['started_at', 'date'], how='left') 
```

</details>

   
Støtte du på en utfordring og fikk en feilmelding her? Hvorfor fungerer ikke dette?

<details><summary>🚨 Løsningsforslag</summary>

Ta en kikk på innholdet i disse to kolonnene. Ser det ut som de er det samme i de ulike tabellene? Her må vi gjøre mer transformasjon før vi kan fortsette!

</details>






### Oppgave 5 - Utvidet datasett 2
Transformasjoner er en stor og viktig del når vi jobber med data. Ofte er datasettene vi har til rådighet ikke på det formatet vi ønsker å ha det på. Å transformere data betyr å gjøre endringer (f.eks slå sammen datasett, endre på datatyper, fjerne duplikater, gjøre utregninger med basis i andre kolonner, eller fjerne potensielle "outliers" som kan ødelegge grunnlaget vårt for analyse)

> 🧼 Dette kalles ofte for å vaske data.

Vi må få datokolonnene til å være på samme format. En måte vi kan gjøre dette på er å kun bruke dato delen av `started_at`. Ulempen med dette er at vi da mister informasjon vi kanskje ønsker å bruke videre i analyse/innsiktsdelen, i dette tilfellet ville vi mistet klokkeslettet. 

Vi løser dette problemet med å lage en hjelpekolonne, altså en ny midlertidg kolonne som kun brukes når vi slår sammen datasettene

1. Lag en ny kolonne, `trip_date`, som kun inneholder datoen fra kolonnen `started_at`.

**💡 Tips**: Pandas har en funksjon [`to_datetime`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) som lar deg tilpasse tidspunkter

<details><summary>🚨 Løsningsforslag</summary>

```
df["trip_date"] = pandas.to_datetime(df["started_at"]).dt.strftime("%Y-%m-%d") 
df.head()

```

</details>



2. Fungerer det å merge nå? 
  - Mest sannsynlig ikke. Hvorfor ikke? 

<details>
<summary>🚨 Løsningsforslag</summary>  
Sjekk datatypene i begge dataframene. Kall `d_types` (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html). Her får du opp hvilke datatyper kolonnene er (det kan være strings, integers, objects ++) Mest sannsynlig består nå kolonnene av object-typer.

</details>

3. For å fortsette må kolonnene ha samme datatype for å kunne slås sammen. Gjør om `trip_date` i bysykkeldatasettet og `date` i værdatasettet til datetime typer.

<details>
<summary>🚨 Løsningsforslag</summary>  
```python
df["trip_date"] = pandas.to_datetime(df["trip_date"])
df_weather["date"] = pandas.to_datetime(df_weather["date"])```

</details>

4. Merge datasettet

<details>
<summary>🚨 Løsningsforslag</summary>  
```
df_merged = df.merge(df_weather, left_on='trip_date', right_on='date', how='left')
df_merged.head()```

</details>

5. Kjør `d_types`. Værdata-kolonnene ser ikke ut til å være av typen tall. Endre datatypen på denne slik at det blir desimaltall

<details>
<summary>🚨 Løsningsforslag</summary>  
Kolonnen ser tilsynelatende ut til å kun bestå av tall. Hvis vi inspiserer verdiene, ser vi at noen ganger forekommer strengen "NULL". Vi må rydde opp i disse før vi kan endre datatypen til float. Vi kan bruke `replace` for dette.

```
df_merged["mean_temperature"] = df_merged["mean_temperature"].replace('NULL',0).astype('float')
df_merged["precipitation_amount"] = df_merged["precipitation_amount"].replace('NULL',0).astype('float')

```

</details>

6. Rydd opp i dataframen ved å fjerne  hjelpekolonnen

<details>
<summary>🚨 Løsningsforslag</summary>  
```
df_merged = df_merged.drop(columns="date")
df_merged.head()
```

</details>

Vi har nå et utvidet datasett! 🎉🥂🎊


Vanligvis ville vi ha skrevet datasettet tilbake til BigQuery, men det gjør vi ikke i denne workshopen. (Det tok lang tid, vi har mye data😴) 

Pandas har en ganske snedig funksjon som kan gjøre dette!

```df_merged.to_gbq("bysykkel_main.bysykkel_med_værdata", project_id="data-intro")```

Når vi skriver til BigQuery, vil datatypene i dataframen følge med og sette riktig skjema i BigQuery
